In [10]:
import json
import math
import os
from pathlib import Path

import gymnasium as gym
import mani_skill2.envs
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from mani_skill2.utils.wrappers import RecordEpisode
from torch.nn import (Flatten, Linear, TransformerEncoder,
                      TransformerEncoderLayer)
from torch.nn.functional import relu
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from data.dataset import StackDatasetOriginalSequential
from utils.data_utils import flatten_obs, make_path
from utils.train_utils import init_deque, update_deque

from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

import os
import h5py
from imitation.data.types import Trajectory




In [11]:
# prepare Trajectory for imitation package
def prep_Trajectory(file_path):
    traj_list = []
    with h5py.File(file_path,'r') as file:
        for traj_key in file.keys():
            traj_data = file[traj_key]
            obs = flatten_obs(traj_data['obs'])
            acts = np.array(traj_data['actions'])
            traj = Trajectory(obs, acts, infos=None,terminal=True) 
            traj_list.append(traj)  
            # print(obs.shape) # (127, 55)
            # print(acts.shape) #(126,8)
            # print(traj_list)
    return traj_list
dir_path = os.getcwd()

data_path = os.path.join(dir_path, '..', 'datasets')
file_path = os.path.join(data_path, 'trajectory_state_original.h5') 

traj_list = prep_Trajectory(file_path)

In [12]:
SEED = 42 

env = gym.make('StackCube-v0',
               obs_mode="state",
               control_mode="pd_joint_delta_pos",
               reward_mode="normalized_dense",
               render_mode="cameras",
               max_episode_steps=250)


[2023-11-16 10:42:41.635] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


In [14]:

learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=traj_list,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
)
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)

# train the learner and evaluate again
gail_trainer.train(800_000)
env.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)



/u/baz7dy/cs6501/NP-RAM/venv/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(


AttributeError: 'StackCubeEnv' object has no attribute 'num_envs'